In [11]:
# import the compulsory
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy import ndimage, linalg
%matplotlib inline

In [4]:
train_input = np.load(os.path.abspath(os.path.curdir) + '/ml_zalando_project/things_we_touch/data/input_train.npy')
train_all = np.load(os.path.abspath(os.path.curdir) + '/ml_zalando_project/things_we_touch/data/OG_files/fashion_train.npy')
train_labels = np.load(os.path.abspath(os.path.curdir) + '/ml_zalando_project/things_we_touch/data/labels_train.npy')
train_shaped = np.load(os.path.abspath(os.path.curdir) + '/ml_zalando_project/things_we_touch/data/dilated_input.npy')

In [24]:
def pca(input_file):
    mean_vector = input_file.mean(axis = 0)
    cov_matrix = np.cov(input_file - mean_vector)
    return linalg.eigh(cov_matrix)

In [25]:
w, v = pca(train_input)

In [28]:
w.min()

-2.6114392930557447e-09